# DCGAN faces - 128x128

Import libraries

In [ ]:
import os
import time
import datetime
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as utils
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from __future__ import print_function
from PIL import Image

%matplotlib inline

Set up paths

In [ ]:
dataset_dir = './dataset/celeba-128/'
figures_dir = './figures/celeba-128/'
checkpoints_dir = './checkpoints/celeba-128/'
graphs_dir = './graphs/celeba-128'

Set up hyperparameters

In [ ]:
workers = 2

batch_size = 128
image_size = 128

nc = 3
noise_dim = 100

nfg = 64
nfd = 64

epochs = 50

g_learning_rate = 0.0002
d_learning_rate = 0.000035
beta1 = 0.5
beta2 = 0.999

True if you want to load the model from disk, False if you want the model to be initialized from scratch

In [ ]:
load_model = False

Set up GPU device for training

In [ ]:
device = torch.device('cuda:0')

Load the dataset

In [ ]:
transform = transforms.Compose([transforms.Resize(image_size),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_data = datasets.ImageFolder(dataset_dir, transform=transform)
data_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size, num_workers=workers)

View samples from the dataset

In [ ]:
ds_sample = next(iter(data_loader))

plt.figure(figsize=(8, 8))
plt.axis('off')
plt.title('Train data')
grid = np.transpose(utils.make_grid(ds_sample[0].to(device)[:64], padding=4, normalize=True).cpu(), (1, 2, 0))
plt.imshow(grid)

Define a method for weights initialization

In [ ]:
def init_weights(model):
    if model.__class__.__name__.find('Conv') != -1:
        nn.init.normal_(model.weight, 0.0, 0.02)
    elif model.__class__.__name__.find('BatchNorm') != -1:
        nn.init.normal_(model.weight, 1.0, 0.02)
        nn.init.zeros_(model.bias)

Define the generator network

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.model = nn.Sequential(
            nn.ConvTranspose2d(noise_dim, nfg*16, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(nfg*16),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(nfg*16, nfg*8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(nfg*8),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(nfg*8, nfg*4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(nfg*4),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(nfg*4, nfg*2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(nfg*2),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(nfg*2, nfg, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(nfg),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(nfg, nc, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )
        
    def forward(self, input):
        return self.model(input)

Define the discriminator network

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.model = nn.Sequential(
            nn.Conv2d(nc, nfd, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Dropout2d(0.5, inplace=False),
            
            nn.Conv2d(nfd, nfd*2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(nfg*2),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(nfd*2, nfd*4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(nfg*4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(nfd*4, nfd*8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(nfg*8),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(nfd*8, nfd*16, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(nfg*16),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Dropout2d(0.5, inplace=False),
            
            nn.Conv2d(nfd*16, 1, kernel_size=4, stride=2, padding=0, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, input):
        return self.model(input)

Instantiate the generator and initialize its weights (option 1)

In [ ]:
if load_model == False:
    generator = Generator().to(device)
    generator.apply(init_weights)

Instantiate the discriminator and initialize its weights (option 1)

In [ ]:
if load_model == False:
    discriminator = Discriminator().to(device)
    discriminator.apply(init_weights)

Load the model from the disk (option 2)

In [ ]:
if load_model == True:
    generator = torch.load(os.path.join(checkpoints_dir, 'generator.pt'))
    discriminator = torch.load(os.path.join(checkpoints_dir, 'discriminator.pt'))

Define the loss function (BinaryCrossEntropy)

In [ ]:
cross_entropy = nn.BCELoss()

Define a noise vector to use to track progress

In [ ]:
sample_noise = torch.randn(64, noise_dim, 1, 1, device=device)

Define the optimizers (Adam)

In [ ]:
disc_optimizer = optim.Adam(discriminator.parameters(), lr=d_learning_rate, betas=(beta1, beta2))
gen_optimizer = optim.Adam(generator.parameters(), lr=g_learning_rate, betas=(beta1, beta2))

Define a function to plot loss

In [ ]:
def plot_loss(gen_losses, disc_losses, epoch=None, save=False, show=True):
    plt.figure(figsize=(10, 5))
    plt.title('Generator and Discriminator losses')
    plt.plot(gen_losses, label='G')
    plt.plot(disc_losses, label='D')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.legend()
    
    if save == True:
        plt.savefig(os.path.join(graphs_dir, f'loss_{epoch}.jpg'))
    if show == True:
        plt.show()

Train both networks simultaneously

In [ ]:
gen_losses = []
disc_losses = []

for epoch in range(epochs):
    start = time.time()
    for i, data in enumerate(data_loader, 0):
        # Train the discriminator
        discriminator.zero_grad()
        
        # Fetch the real data
        real = data[0].to(device)
        size = real.size(0)
        
        # Init labels as ones for real data
        label = torch.full((size,), 1, device=device, dtype=torch.float)
        output = discriminator(real).view(-1)
        
        # Compute loss for real images
        disc_err_real = cross_entropy(output, label)
        disc_err_real.backward()
        real_mean = output.mean().item()
        
        # Train the discriminator on fake images
        noise = torch.randn(size, noise_dim, 1, 1, device=device)
        fake = generator(noise)
        
        # Init labels as zeros for fake data
        label.fill_(0)
        output = discriminator(fake.detach()).view(-1)
        
        # Compute loss for fake images
        disc_err_fake = cross_entropy(output, label)
        disc_err_fake.backward()
        fake_mean = output.mean().item()
        disc_err = disc_err_real + disc_err_fake
        disc_optimizer.step()
        
        # Train the generator
        generator.zero_grad()
        
        # Init labels as ones
        label.fill_(1)
        noise = torch.randn(size, noise_dim, 1, 1, device=device)
        fake = generator(noise)
        output = discriminator(fake).view(-1)
        
        # Compute generator loss
        gen_err = cross_entropy(output, label)
        gen_err.backward()
        gen_mean = output.mean().item()
        gen_optimizer.step()
        
        if i % 100 == 0:
            print('[%d/%d][%d/%d] \tD-Loss:%.4f\t G-Loss:%.4f\t D(x):%.4f\t D(G(z)):%.4f\t G(z):%.4f' 
                  % (epoch + 1, epochs, i + 1, len(data_loader), disc_err.item(), gen_err.item(), real_mean, fake_mean, gen_mean))
            
        gen_losses.append(gen_err.item())
        disc_losses.append(disc_err.item())
        
    end = time.time()
    timedelta = datetime.timedelta(seconds=int(end - start))
    print(f'Time elapsed for epoch {epoch + 1}: {timedelta}\n')
    with torch.no_grad():
        sample = generator(sample_noise).detach().cpu()
    grid = np.transpose(utils.make_grid(sample, padding=4, normalize=True).cpu(), (1, 2, 0))
    
    # Generate samples after every epoch
    plt.figure(figsize=(8, 8))
    plt.axis('off')
    plt.imshow(grid)
    plt.savefig(os.path.join(figures_dir, f'epoch_{epoch + 1}.png'))
    plt.close()
    
    if epoch % 10 == 0:
        # Generate loss graph
        plot_loss(gen_losses, disc_losses, epoch + 1, save=True, show=False)
        
        # Save progress
        torch.save(generator, os.path.join(checkpoints_dir, f'generator.pt'))
        torch.save(discriminator, os.path.join(checkpoints_dir, f'discriminator.pt'))

Clear CUDA cache if needed

In [ ]:
# torch.cuda.empty_cache()

Plot the loss graph

In [ ]:
plot_loss(gen_losses, disc_losses, epoch='final', save=True, show=True)

Generate samples on random noise

In [ ]:
noise = torch.randn(25, noise_dim, 1, 1, device=device).detach()
with torch.no_grad():
    sample = generator(noise).detach().cpu()
grid = np.transpose(utils.make_grid(sample, padding=4, normalize=True, nrow=5).cpu(), (1, 2, 0))
plt.figure(figsize=(5, 5))
plt.axis('off')
plt.imshow(grid)
# plt.savefig(os.path.join(figures_dir, f'random_{i}.jpg'))
# plt.close()